In [47]:
import json
import re
import requests
import os
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np
from IPython.display import display, HTML


# Run ColBERT Reranker
from primeqa.components.reranker.colbert_reranker import ColBERTReranker
model_name_or_path = "/Users/abhilashamangal/Documents/Hanzo/PrimeQA/article/DrDecr.dnn"

llmToken = os.getenv('LLM_TOKEN')


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

In [48]:
def skip_unwanted_characters(document, keyword):
        lines = document.split('\n')
        desired_text = ""
        last_occurrence = -1
        for i, line in enumerate(lines):
            if keyword in line:
                last_occurrence = i
                
        if last_occurrence != -1:
            for line in lines[last_occurrence+1:]:
                desired_text += line.strip() + "\n"
        else:
            desired_text = document
    
        return desired_text.strip()

In [106]:
def pre_processingtext(text_data):
    replaced = re.sub("\{{ .*?\}}", "", text_data)
    replaced = re.sub("\{: .*?\}", "", replaced)
    replaced = re.sub("\.*?", "", replaced)
    replaced = re.sub("\(.*?\)|\[.*?\] |\{.*?\}", "", replaced)
    replaced = re.sub("</?div[^>]*>", "", replaced)
    replaced = re.sub("</?p[^>]*>", "", replaced)
    replaced = re.sub("</?a[^>]*>", "", replaced)
    replaced = re.sub("</?h*[^>]*>", "", replaced)
    replaced = re.sub("</?em*[^>]*>", "", replaced)
    replaced = re.sub("</?img*[^>]*>", "", replaced)
    replaced = re.sub("&amp;", "", replaced)
    replaced = re.sub("</?href*>", "", replaced)
    replaced = re.sub("\s+", " ", replaced)
    replaced = replaced.replace("}","")
    replaced = replaced.replace("##","")
    replaced = replaced.replace("###","")
    replaced = replaced.replace("#","")
    replaced = replaced.replace("*","")
    replaced = replaced.replace("<strong>","")
    replaced = replaced.replace("</strong>","")
    replaced = replaced.replace("<ul>","")
    replaced = replaced.replace("</ul>","")
    replaced = replaced.replace("<li>","")
    replaced = replaced.replace("</li>","")
    replaced = replaced.replace("<ol>","")
    replaced = replaced.replace("</ol>","")
    return replaced


In [125]:
# Retrieve documents
max_num_documents=10

def solr_reteriver(question):
    question = question.replace("?","")
    response = requests.get(f'http://150.239.171.68:8983/solr/superknowa/select?q='+question+'&q.op=AND&wt=json')
    query_result = response.json()
    
   # print("SOLR RESPONSE:", json.dumps(query_result, indent=2))
    print(query_result['response']['numFound'], "documents found.")
    total = query_result['response']['numFound']
    results_list=[]
    query_hits={}
    if total > 0:
        if total > 10:
            total =10
        for i in range(total):
                string_unicode = query_result['response']['docs'][i]['content'][0]
                doc = string_unicode.encode("ascii", "ignore")
                string_decode = doc.decode()
                keyword = "{: shortdesc} "
                cleaned_text = skip_unwanted_characters(string_decode, keyword)
                pattern =  r'\{\s*:\s*[\w#-]+\s*\}|\{\s*:\s*\w+\s*\}|\n\s*\n'
                cleaned_text = re.sub(pattern, '', cleaned_text)
                cleaned_text = pre_processingtext(cleaned_text)
                query_hits = {
                "document": {
                    "rank": i,
                    "document_id": query_result['response']['docs'][i]['id'][0],
                    "text": cleaned_text[0:4000], 
                    "url" :query_result['response']['docs'][i]['url'][0].replace(" ","")
                },
            }

                results_list.append(query_hits)
                results_to_display = [results_list['document'] for results_list in results_list]
                df = pd.DataFrame.from_records(results_to_display, columns=['rank','document_id','text','url'])
                # df['title'] = np.random.randint(1, 10, df.shape[0])
                df.dropna(inplace=True)
                print('======================================================================')             
    print(f'QUERY: {question}')
    return results_list

In [127]:
results_list=solr_reteriver("what is ibm cloud pak for data")
print(results_list)

82 documents found.
QUERY: what is ibm cloud pak for data
[{'document': {'rank': 0, 'document_id': ' ', 'text': 'It used to be that the developer and operations roles were fully separated. The developer received the business requirements, they came up with a design and implemented it. A dedicated QA team wrote tests, and validated that the new application, capability, or change worked as expected, and then the code or application was handed over to the operations team to run it. In most organizations, these roles have now merged into a set of development practices that are commonly referred to as DevOps.\nTo learn more about what is SRE, read this Red Hat article.\nAt the same time, DevOps practices brought an expanded role for the operations teams, typically referred to now as a Site Reliability Engineering, or SRE. Red Hat defines the DevOps practice of SRE like this SRE takes the tasks that have historically been done by operations teams, often manually, and instead gives them to en

In [128]:
def solr_reranker(question, max_reranked_documents = 10):

    reranker = ColBERTReranker(model=model_name_or_path)
    reranker.load()
    
    results_list = solr_reteriver(question)
    if len(results_list) >0:
        reranked_results = reranker.predict(queries= [question], documents = [results_list], max_num_documents=max_reranked_documents)

        print(reranked_results)

        reranked_results_to_display = [result['document'] for result in reranked_results[0]]
        df = pd.DataFrame.from_records(reranked_results_to_display, columns=['rank','document_id','text','url'])
        print('======================================================================')
        print(f'QUERY: {question}')
        display( HTML(df.to_html()) )
        return df['text'][0] , df['url'][0]
    else:
        return "0 documents found" , "None"

In [129]:
#solr_reranker("What is IBM Cloud ?")

In [130]:
def format_string(doc):
    doc = doc.encode("ascii", "ignore")
    string_decode = doc.decode()
    cleantext = BeautifulSoup(string_decode, "lxml").text
    perfecttext = " ".join(cleantext.split())
    perfecttext = re.sub(' +', ' ', perfecttext).strip('"')
    #perfecttext = perfecttext[0:4000]
    return perfecttext

In [131]:
def process_llm_request(question):
    
    wd_result,url = solr_reranker(question)
    if '0 documents found' not in wd_result:
        combined_input = "Answer the question based only on the context below. " + \
            "Context: "  + format_string(wd_result) + \
            " Question: " + question
        print("INPUT PROMPT: ", combined_input)

        headers = {
            'Content-Type': 'application/json',
            'Authorization': llmToken,
        }

        json_data = {
            'model_id': 'bigscience/bloom',
            'inputs':  [combined_input],        
                'parameters': {
                # "stream": "true",
                'temperature': 0.5,
                'max_new_tokens': 200,
            },
        }

        ## Demo LINK of llm 
        response = requests.post('https://llm-api.res.demo.ibm.com/v1/generate', headers=headers, json=json_data)
        json_response = json.loads(response.content.decode("utf-8"))
        result = json_response['results'][0]['generated_text'].split("Answer:")
        if len(result) > 1:
            print("LLM Output: ", result[1])
            return result[1],url
        else:
            print("LLM Output: ", result[0])
            return result[0],url
    else:
        return "0 documents found" , "None"

In [132]:
def main():
    question = "what is ibm cloud pak for data"
    # print("-------- Final answer ---------------")
    answer ,url = process_llm_request(question)
    print("FINAL ANSWER: ", answer)
    print("URL: ", url)
    return answer , url

In [133]:
main()

[May 23, 01:19:20] #>>>>> at ColBERT name (model type) : /Users/abhilashamangal/Documents/Hanzo/PrimeQA/article/DrDecr.dnn
[May 23, 01:19:20] #>>>>> at BaseColBERT name (model type) : /Users/abhilashamangal/Documents/Hanzo/PrimeQA/article/DrDecr.dnn
[May 23, 01:19:23] factory model type: xlm-roberta-base
[May 23, 01:19:35] get query model type: xlm-roberta-base
[May 23, 01:19:39] get doc model type: xlm-roberta-base


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


82 documents found.
QUERY: what is ibm cloud pak for data
[May 23, 01:19:46] #> XMLR QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
[May 23, 01:19:46] #> Input: $ what is ibm cloud pak for data, 		 True, 		 None
[May 23, 01:19:46] #> Output IDs: torch.Size([32]), tensor([    0,  9748,  2367,    83,     6,  2566,    39, 76746,  2522,   100,
         2053,     2,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1])
[May 23, 01:19:46] #> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
[May 23, 01:19:46] #>>>> colbert query ==
[May 23, 01:19:46] #>>>>> input_ids: torch.Size([32]), tensor([    0,  9748,  2367,    83,     6,  2566,    39, 76746,  2522,   100,
         2053,     2,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1, 

[May 23, 01:19:46] #> Output Mask: torch.Size([180]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
[May 23, 01:19:46] #>>>> colbert doc ==
[May 23, 01:19:46] #>>>>> input_ids: torch.Size([180]), tensor([     0,   9749,   1650,  11814,     47,    186,    450,     70, 106001,
           136,  41018,      7,  31486,      7,   3542,  89554,  84797,     71,
             5,    581, 106001,  75204,     70,   8063,  96679,      4,   1836,
     

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


[May 23, 01:19:49] #>>>> before linear doc ==
[May 23, 01:19:49] #>>>>> D: torch.Size([180, 768]), tensor([[ 0.0963,  0.1672,  0.2844,  ..., -0.0999,  0.1511,  0.2975],
        [-0.2177, -0.1156,  0.1032,  ..., -0.2990,  0.0331,  0.4527],
        [-0.1852, -0.0389,  0.1935,  ...,  0.1389,  0.0033,  0.4150],
        ...,
        [-0.0796, -0.1400,  0.2745,  ...,  0.6648,  0.1962,  0.6829],
        [-0.1573, -0.2491,  0.2658,  ...,  0.3211,  0.2732,  0.9019],
        [ 0.0965,  0.1657,  0.2779,  ..., -0.1104,  0.1406,  0.3077]])
[May 23, 01:19:49] #>>>>> self.linear doc : Parameter containing:
tensor([[-0.0286,  0.0017, -0.0202,  ..., -0.0262,  0.0210,  0.0006],
        [-0.0102,  0.0121, -0.0111,  ..., -0.0362, -0.0165, -0.0012],
        [-0.0047, -0.0172, -0.0054,  ..., -0.0069, -0.0194, -0.0193],
        ...,
        [-0.0286,  0.0231,  0.0004,  ...,  0.0373, -0.0045,  0.0125],
        [ 0.0051,  0.0023,  0.0212,  ..., -0.0254,  0.0034,  0.0206],
        [-0.0068,  0.0256, -0.0263,  .

,rank,document_id,text,url
0,7,,"""Awww, come on guys, it's so simple. Maybe you need a refresher course. ... It's all ball bearings nowadays."" - Fletch, 1985 movie\nSadly, it is not all about ball bearings nowadays. It's all about containers. If you heard about containers, but are not sure what they are, you've come to the right place. This blog post addresses the following questions:Why should I care about containers?\nWhat are containers?\nAre containers the same as microservices?\nWhat is an example of a microservices application?\nWhat is a Docker container?\nWhat are container orchestration and Kubernetes?\nWhat is the difference between containers and virtual machine images?\nHow can I get started with containers?\nHow can IBM Cloud Paks help?\nWhere can I run my containers?\nWhat is Red Hat OpenShift on IBM Cloud?Why should I care about containers?\nAcross organizations, there is a spectrum of container adoption. Many people are just learning about containers. Some companies are further along in their journey. If you're at all considering containerization, it's time to join the fun where you can see real business results:Faster time to market: New applications and services are what keep your competitive edge. Organizations are able to speed up delivery of new services with development and operational agility.\nDeployment velocity: Move quicker from development to deployment. Containerization breaks down barriers for DevOps teams to accelerate deployment times and frequency.\nIT infrastructure reduction: Reduce your costs by increasing your application workload density, getting better utilization of your server compute density, and reducing software licensing costs.\nIT operational efficiency: Gain more operational efficiency by streamlining and automating the management of diverse applications and infrastructure into a single operating model.\nGain freedom of choice: Package, ship, and run applications on any public or private cloud.Next steps: Learn what the true benefits of moving to containers are.What are containers?\nThe best analogy for understanding containers is a shipping container. That's why the majority of all container articles and blog posts show a photo of a shipping container. We're sure you've seen the transport of those big steel shipping containers. The shipping industry standardized on a consistent size container. Now, the same container can move from a ship to a train to a truck without unloading the cargo. The container contents do not matter.\nJust like a shipping container, a software container is a standardized package of software. Everything needed for the software to run is inside the container. The software code, runtime, system tools, system libraries, and settings are all inside a single container.\nAre containers the same thing as microservices?\nOnce you start diving into containers, it's impossible to avoid reading about microservices. Microservices is an architectural style. A microservices architecture structures an application by using as a collection of loosely coupled services, which deliver specific business capabilities. Containers help make it happen.\nWhat is an example of a microservices application?\nMore than ten years ago, Netflix was one of the first companies to begin using containers extensively. They rewrote the applications that ran their entire video service by using a microservices architecture. In 2017, Netflix estimated that it employed around 700 microservices to control each of the many functions that make up its service. Let's look at a few :Video selection: A microservice, in a container, provides your phone, tablet, computer, or TV with the video file to play and at a video quality based on your internet speed.\nViewing history: One microservice remembers what shows you watch.\nProgram recommendations: A microservice takes a look at your viewing history and uses analytics to recommend movies.\nMain menu: One microservice provides the names and images of these

INPUT PROMPT:  Answer the question based only on the context below. Context: Awww, come on guys, it's so simple. Maybe you need a refresher course. ... It's all ball bearings nowadays." - Fletch, 1985 movie Sadly, it is not all about ball bearings nowadays. It's all about containers. If you heard about containers, but are not sure what they are, you've come to the right place. This blog post addresses the following questions:Why should I care about containers? What are containers? Are containers the same as microservices? What is an example of a microservices application? What is a Docker container? What are container orchestration and Kubernetes? What is the difference between containers and virtual machine images? How can I get started with containers? How can IBM Cloud Paks help? Where can I run my containers? What is Red Hat OpenShift on IBM Cloud?Why should I care about containers? Across organizations, there is a spectrum of container adoption. Many people are just learning about

(' IBM Cloud Pak for Data is a purpose-built, integrated platform that provides the foundation for modernizing data and analytics workloads. It is a fully open, extensible, and integrated platform that brings together the best of IBM and Red Hat technologies, including Red Hat OpenShift on IBM Cloud, IBM Cloud Pak for Data, IBM Cloud Pak for Integration, and IBM Cloud Pak for Security. IBM Cloud Pak for Data is a platform for modernizing data and analytics. You can use it to manage the data lifecycle, from data ingestion to data storage to data processing to data visualization to data governance. IBM Cloud Pak for Data is an open and integrated platform that brings together the best of IBM and Red Hat technologies. It includes the following capabilities:IBM Cloud Pak for Data is a platform for modernizing data and analytics. You can use it to manage the data lifecycle, from data ingestion to data storage to data processing to data visualization to data governance.',
 'https://developer